In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import json

In [2]:
master_list = pd.read_csv("..//data/searchResults.csv")

## Remove extra rows


In [3]:
def find_bad_row(row, col, term):
    if re.search(term, row[col], flags = re.I) is not None:
        return True
    return False

In [4]:
bad_indexes = master_list.apply(lambda row: find_bad_row(row, "License Number", "license number"), axis = 1)
master_list = master_list[~bad_indexes].reset_index(drop = True)

## Add extra fields

In [5]:
def break_fields(row, col, string):
    if pd.isnull(row[col]):
        return ""
    
    test = re.search(string, row[col], flags = re.I)
    if test != None:
        return test.group()
    
    return ""

def get_city(row, col):
    street_abbrev = [" w ", " blvd ", " st ", " rd ", " pkwy ", " ave ", 
                     " ctr ", " cir ", " ct ", " dr ", " ln ", " lk ", 
                     " lp ", " pl ", " sq ", " tr ", " e ", " n ", " s ", 
                     " hwy ", " way ", " wy ", ]
    fix = {"[0-9]": "", "\s+": " ", "san fran": "san francisco", "avenue": "", "highway": ""}
    if pd.isnull(row[col]):
        return ""
    
    left_limit = 0
    check_list = []
    for i in street_abbrev:
        street = re.search(i, row[col], flags = re.I)
        if street != None:  
            check_list.append(street.end())
        else:
            check_list.append(0)
    left_limit = max(check_list)
    
    right_limit = re.search(", ", row[col], flags = re.I)
    if right_limit != None:
        right_limit = right_limit.start()
        
    tmp = row[col][left_limit: right_limit].strip()
    
    for i in fix:
        tmp = re.sub(i, fix[i], tmp, flags = re.I)
        
    return tmp.strip()

def get_city(row, col):
    street_abbrev = [" w ", " blvd ", " st ", " rd ", " pkwy ", " ave ", 
                     " ctr ", " cir ", " ct ", " dr ", " ln ", " lk ", 
                     " lp ", " pl ", " sq ", " tr ", " e ", " n ", " s ", 
                     " hwy ", " way ", " wy ", ]
    fix = {"[0-9]": "", "\s+": " ", "san fran": "san francisco", "avenue": "", "highway": "", "san franciscocisco": "san francisco"}
    if pd.isnull(row[col]):
        return ""
    
    left_limit = 0
    check_list = []
    for i in street_abbrev:
        street = re.search(i, row[col], flags = re.I)
        if street != None:  
            check_list.append(street.end())
        else:
            check_list.append(0)
    left_limit = max(check_list)
    
    right_limit = re.search(", ", row[col], flags = re.I)
    if right_limit != None:
        right_limit = right_limit.start()
        
    tmp = row[col][left_limit: right_limit].strip()
    
    for i in fix:
        tmp = re.sub(i, fix[i], tmp, flags = re.I)
        
    return tmp.strip()

def get_email(row, col):
    replace = {"email": "", "Email": "", "-": ""}
    if pd.isnull(row[col]):
        return ""
    check = re.search("email.{,1}?-.+?@.+?\s", row[col], flags = re.I)
    
    if check is not None:
        check = check.group()
        for i in replace:
            check = check.replace(i, replace[i])
        return check.strip()
    
    return ""

def get_website(row, col):
    if pd.isnull(row[col]):
        return ""
    check = re.search("website-", row[col], flags = re.I)
    
    if check is not None:
        check = row[col][check.end():]
        return check.strip()
    
    return ""

def get_name(row, col):
    if pd.isnull(row[col]):
        return ""
    check = re.search(":", row[col], flags = re.I)
    
    if check is not None:
        check = row[col][:check.end() - 1]
        return check.strip()
    
    return ""

def get_phone(row, col):
    if pd.isnull(row[col]):
        return ""
    
    check = re.search("phone-.+?\s", row[col], flags = re.I)
    #print(check)
    
    if check is not None:
        check = re.sub("-|:|\(|\)|phone", "", check.group(), flags = re.I)
        return check.strip()
        
    return ""

In [6]:
master_list["zip_code"] = master_list.apply(lambda row: break_fields(row, "Premise Address", "([0-9]{5})|([0-9]{9})"), axis = 1)
master_list["city"] = master_list.apply(lambda row: get_city(row, "Premise Address"), axis = 1).str.lower()
master_list["email"] = master_list.apply(lambda row: get_email(row, "Business Contact Information"), axis = 1).str.lower()
master_list["website"] = master_list.apply(lambda row: get_website(row, "Business Contact Information"), axis = 1).str.lower()
master_list["company_name"] = master_list.apply(lambda row: get_name(row, "Business Contact Information"), axis = 1).str.lower()
master_list["phone"] = master_list.apply(lambda row: get_phone(row, "Business Contact Information"), axis = 1).astype(str)
master_list["License_no_dash"] = master_list["License Number"].str.replace("-", "")
#master_list["master_index"] = master_list.index

In [7]:
master_list.to_csv("..//data/searchResultsClean.csv", index = False)

In [14]:
retailers = master_list[master_list["License Type"].isin(["Cannabis - Retailer Temporary License", "Cannabis - Retailer Nonstorefront Temporary License"])]

In [12]:
from collections import Counter

Counter({'Active': 942, 'Canceled': 74, 'Expired': 149, 'Revoked': 3})

In [17]:
retailers = retailers[retailers.Status == "Active"]

In [19]:
retailers.to_csv("..//data/searchResultsRetailers.csv", index = False)